# Basic Message - Bob

**Should be run alongside Alice.**

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(admin_url=ADMIN_URL)

In [3]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## Setup a listener for the basicmessages topic

This is emitted using PyPubSub when the controller receives a basicmessages webhook from the agent. This happens everytime the agent receives a basicmessage.

Note, in this instance the handler responds to any basic message with a message of it's own.

In [4]:
def messages_handler(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, "This is a response from Bob"))
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

Subscribing too: basicmessages


## Verify an Active Connection Exists

This should have been established as part of the setup script between the two agents Alice and Bob.

In [5]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'created_at': '2023-01-01 15:38:32.965206Z', 'invitation_key': 'C9VVc5HS8ASbZGJ4RRH38huh7jxzV1Lxc6dv9MMhYRm6', 'state': 'active', 'request_id': '5ef4cc1f-c5f0-4a09-8bf0-49631cd28b22', 'their_did': '78JbknZS4Un2nRsSXthrrX', 'my_did': '6UJzVcKRUTNd2Z3fkgSwqc', 'rfc23_state': 'completed', 'updated_at': '2023-01-01 15:38:46.632967Z', 'their_role': 'inviter', 'accept': 'manual', 'invitation_mode': 'once', 'connection_id': '9777a4c5-00c9-4816-acd3-3139431cee8b', 'routing_state': 'none', 'their_label': 'Alice'}]
Connection : {'created_at': '2023-01-01 15:38:32.965206Z', 'invitation_key': 'C9VVc5HS8ASbZGJ4RRH38huh7jxzV1Lxc6dv9MMhYRm6', 'state': 'active', 'request_id': '5ef4cc1f-c5f0-4a09-8bf0-49631cd28b22', 'their_did': '78JbknZS4Un2nRsSXthrrX', 'my_did': '6UJzVcKRUTNd2Z3fkgSwqc', 'rfc23_state': 'completed', 'updated_at': '2023-01-01 15:38:46.632967Z', 'their_role': 'inviter', 'accept': 'manual', 'invitation_mode': 'once', 'connection_id': '9777a4c5-00c9-4816-acd3-3139431cee8b', '

## Send a Basic Message over DIDComm to Bob

See [aries-rfc](https://github.com/hyperledger/aries-rfcs/tree/master/features/0095-basic-message)

You can send as many messages as you want, if you are running the [basic-message tutorial on Alice's notebook](http://localhost:8888/lab/tree/3%20Messages/Part%201%20-%20Basic%20Message.ipynb) these will be received and printed by the message handler.

In [7]:
basic_message = "Hello Alice"
response = await agent_controller.messaging.send_message(connection_id, basic_message)
print("BASIC MESSAGE - Bob -> Alice")
print(response)

BASIC MESSAGE - Bob -> Alice
{}
Handle message {'connection_id': '9777a4c5-00c9-4816-acd3-3139431cee8b', 'message_id': 'ad2d4a13-0595-44f9-a6c7-4206c6897c14', 'content': 'hello Bob', 'state': 'received'} 9777a4c5-00c9-4816-acd3-3139431cee8b


## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [8]:
response = await agent_controller.terminate()
print(response)

None
